#### [ 미니 테스트  2024-03-14 ]<hr>

[1] 지도학습에 대한 설명과 종류에 대해서 간략하게 설명해 주세요.

데이터와 라벨을 주고 정답을 알려주면서 기계를 학습하는 방법, 종류에는 회귀와 분류가 있다.

[2] 지도학습의 학습 프로세스를 간략하게 설명해 주세요.

데이터확인(상관계수 및 시각화) -> 데이터 전처리(인코딩, 범주화, 스케일링, 이상치-중복치-결측치 제거 or 대체) -> 데이터셋 나누기 -> 모델 학습 -> 모델 평가 -> 예측 -> 활용

[3] 선형회귀 대해 간략하게 설명해주세요.  
    - 개념  
    - 조건  
    - 동작원리  
    - 선형식  
    - 오차/잔차 계산 방법  

- 선형회귀는 데이터에 대한 선형 관계를 가정하여 데이터를 예측하는 모델
- 상관계수를 봤을때, 상관계수가 높거나,  산점도를 확인 했을때, 데이터가 선형적으로 분포되어 있어야 한다.
- 오차가 가장 적은 선을 찾는다.
- 선형식 : 예측값 = 가중치 * 입력값 + 절편
- 오차/잔차 계산 방법 : 오차 = 타겟값 - 예측

[4] 로지스틱회귀 대해 간략하게 설명해주세요.  
   - 개념  
   - 조건  
   - 동작원리  
   - 선형식  
   - 오차/잔차 계산 방법  

- 로지스틱회귀는 분류 문제를 해결하기 위한 회귀 모델 : 2진 분류 / 0 아니면 1
- 타겟의 개수가 2개이고, 예측값이 0과 1 사이여야 한다. 
- 
- 
- 타겟이 1일때 오차값 + 타겟이 0일때 오차값 / 오차함수가 총 두개

[5] Fashion MNIST 데이터셋에서 10개 카테고리 중 하나를 선택해서 2진 분류로 구현해 주세요.  
- 데이터셋 : scikit-learn의 datasets 사용
    - from sklearn.datasets import fetch_openml 
    - db_name = 'Fashion-MNIST'
    - fashion_data = fetch_openml(name=db_name, parser='auto')

In [61]:
from sklearn.datasets import fetch_openml
import pandas as pd
import torch
import torch.nn as nn      # LinearRegressoion 기능의 클래스 Linear
import torch.nn.functional as F
import torch.optim as optim
from sklearn.preprocessing import OneHotEncoder

In [62]:
fashion_data = fetch_openml(name='Fashion-MNIST', parser='auto',as_frame=True) 

In [64]:
# 정규화 ==> 피쳐 정규화 : 784개는 색상값 즉, 0 ~ 255 범위의 값
feature = fashion_data.data / 255.   # 실수로 나누기 ! => 정수/실수 = 실수 => 형변환
target = fashion_data.target

In [65]:
feature.shape, target.shape

((70000, 784), (70000,))

In [66]:
target = target.astype('float')

In [67]:
target.value_counts() # 0과 나머지

class
9.0    7000
0.0    7000
3.0    7000
2.0    7000
7.0    7000
5.0    7000
1.0    7000
6.0    7000
4.0    7000
8.0    7000
Name: count, dtype: int64

In [68]:
# 다중분류  ==> 2진분류 변환 : target 변환
target.replace([1,2,3,4,5,6,7,8,9],1, inplace = True)
target.value_counts()

class
1.0    63000
0.0     7000
Name: count, dtype: int64

In [74]:
xtrain = feature[:60000]
ytrain = target[:60000]

xtest = feature[60000:]
ytest = target[60000:]
ytrain, ytest

(0        1.0
 1        0.0
 2        0.0
 3        1.0
 4        0.0
         ... 
 59995    1.0
 59996    1.0
 59997    1.0
 59998    0.0
 59999    1.0
 Name: class, Length: 60000, dtype: float64,
 60000    1.0
 60001    1.0
 60002    1.0
 60003    1.0
 60004    1.0
         ... 
 69995    1.0
 69996    1.0
 69997    1.0
 69998    1.0
 69999    1.0
 Name: class, Length: 10000, dtype: float64)

In [75]:
# 텐서로 바꾸기
xtrain = torch.tensor(xtrain.values, dtype = torch.float32)
xtest = torch.tensor(xtest.values, dtype = torch.float32)

In [76]:
xtrain.shape, xtrain.ndim

(torch.Size([60000, 784]), 2)

In [77]:
ytrain = torch.tensor(ytrain.values, dtype = torch.float32)
ytrain = ytrain.unsqueeze(dim = 1)
ytest = torch.tensor(ytest.values, dtype = torch.float32)
ytest = ytest.unsqueeze(dim = 1)

In [78]:
ytrain.shape, ytest.shape

(torch.Size([60000, 1]), torch.Size([10000, 1]))

In [82]:
model = nn.Sequential(nn.Linear(784,1), nn.Sigmoid())

# optimizer 설정
optimizer = optim.SGD(model.parameters(), lr= 0.3)

In [80]:
epochs = 10000
min_loss = 100.0
for epoch in range(1+epochs):
    ypre = model(xtrain)

    loss = F.binary_cross_entropy(ypre, ytrain)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if epoch % 10 == 0:

        pre = ypre >= torch.FloatTensor([0.5])   
        pre_1 = pre.float() == ytrain  # list
        accuracy = pre_1.sum().item() / len(pre_1) # 정확도를 계산

        print(f'Epoch {epoch:4d}/{epochs} Cost: {loss.item()} Accuracy {accuracy * 100:2.2f}%')
        if loss < min_loss:
            min_loss = loss
        else:
            print('학습 종료')
            break

Epoch    0/10000 Cost: 0.6572571992874146 Accuracy 69.68%
Epoch   10/10000 Cost: 0.19497975707054138 Accuracy 94.77%
Epoch   20/10000 Cost: 0.14314012229442596 Accuracy 95.35%
Epoch   30/10000 Cost: 0.13381953537464142 Accuracy 95.40%
Epoch   40/10000 Cost: 0.12961874902248383 Accuracy 95.44%
Epoch   50/10000 Cost: 0.12686114013195038 Accuracy 95.46%
Epoch   60/10000 Cost: 0.1248626783490181 Accuracy 95.48%
Epoch   70/10000 Cost: 0.12331392616033554 Accuracy 95.49%
Epoch   80/10000 Cost: 0.12205559760332108 Accuracy 95.49%
Epoch   90/10000 Cost: 0.12099768966436386 Accuracy 95.49%
Epoch  100/10000 Cost: 0.12008541822433472 Accuracy 95.49%
Epoch  110/10000 Cost: 0.11928333342075348 Accuracy 95.50%
Epoch  120/10000 Cost: 0.11856749653816223 Accuracy 95.51%
Epoch  130/10000 Cost: 0.11792097985744476 Accuracy 95.51%
Epoch  140/10000 Cost: 0.1173313781619072 Accuracy 95.52%
Epoch  150/10000 Cost: 0.11678938567638397 Accuracy 95.51%
Epoch  160/10000 Cost: 0.11628790199756622 Accuracy 95.52%


In [81]:
# 예측
ypre = model(xtest[0]) # 1일 확신도가 0.999757
ypre = ypre.detach().numpy()[0]
if ypre >= 0.5:
    yp = 1
else:
    yp = 0
print(f'확신도 : {ypre} / 예측값 : {yp} / 실제값 : {ytest[0].type(torch.int).item()}')

확신도 : 0.9999345541000366 / 예측값 : 1 / 실제값 : 1
